# Ejercicio: entrenamiento de un modelo de regresión lineal múltiple
En este ejercicio, entrenaremos un modelo de regresión lineal simple y un modelo de regresión lineal múltiple, y compararemos su desempeño usando R-Squared. 
## Cargando datos 
Comencemos echando un vistazo a nuestros datos.

In [1]:
import pandas

#!wget https://raw.githubusercontent.com/MicrosoftDocs/mslearn-introduction-to-machine-learning/main/graphing.py
#!wget https://raw.githubusercontent.com/MicrosoftDocs/mslearn-introduction-to-machine-learning/main/Data/doggy-illness.csv

#Import the data from the .csv file
dataset = pandas.read_csv('doggy-illness.csv', delimiter="\t")

#Let's have a look at the data
dataset

,male,attended_training,age,body_fat_percentage,core_temperature,ate_at_tonys_steakhouse,needed_intensive_care,protein_content_of_last_meal
0,0,1,6.9,38,38.423169,0,0,7.66
1,0,1,5.4,32,39.015998,0,0,13.36
2,1,1,5.4,12,39.148341,0,0,12.90
3,1,0,4.8,23,39.060049,0,0,13.45
4,1,0,4.8,15,38.655439,0,0,10.53
...,...,...,...,...,...,...,...,...
93,0,0,4.5,38,37.939942,0,0,7.35
94,1,0,1.8,11,38.790426,1,1,12.18
95,0,0,6.6,20,39.489962,0,0,15.84
96,0,0,6.9,32,38.575742,1,1,9.79


Para este ejercicio, intentaremos predecir la temperatura central a partir de algunas de las otras funciones disponibles. 
## Visualización de datos 
Veamos rápidamente qué características parecen tener algún tipo de relación con core_temperature

In [2]:
import graphing # Custom graphing code that uses Plotly. See our GitHub repository for details

graphing.box_and_whisker(dataset, "male", "core_temperature", show=True)
graphing.box_and_whisker(dataset, "attended_training", "core_temperature", show=True)
graphing.box_and_whisker(dataset, "ate_at_tonys_steakhouse", "core_temperature", show=True)
graphing.scatter_2D(dataset, "body_fat_percentage", "core_temperature", show=True)
graphing.scatter_2D(dataset, "protein_content_of_last_meal", "core_temperature", show=True)
graphing.scatter_2D(dataset, "age", "core_temperature")

A simple vista, los perros más gordos, viejos y machos parecen tener temperaturas más altas que los perros más delgados, jóvenes o hembras. Los perros que comieron muchas proteínas la noche anterior también parecen encontrarse peor. Las demás características no parecen especialmente útiles.

## Regresión lineal simple
Intentemos predecir la temperatura_del_núcleo mediante regresión lineal simple y observemos la R-cuadrado de estas relaciones.

In [3]:
import statsmodels.formula.api as smf
import graphing # custom graphing code. See our GitHub repo for details

for feature in ["male", "age", "protein_content_of_last_meal", "body_fat_percentage"]:
    # Perform linear regression. This method takes care of
    # the entire fitting procedure for us.
    formula = "core_temperature ~ " + feature
    simple_model = smf.ols(formula = formula, data = dataset).fit()

    print(feature)
    print("R-squared:", simple_model.rsquared)
    
    # Show a graph of the result
    graphing.scatter_2D(dataset, label_x=feature, 
                                 label_y="core_temperature",
                                 title = feature,
                                 trendline=lambda x: simple_model.params[1] * x + simple_model.params[0],
                                 show=True)

male
R-squared: 0.0999007443071992


age
R-squared: 0.2648116081342463


protein_content_of_last_meal
R-squared: 0.9155158150005704


body_fat_percentage
R-squared: 0.00020809002637678375


Al desplazarnos por estos gráficos, obtenemos valores R-cuadrado de 0,0002 (porcentaje_grasa_corporal), 0,1 (varón) y 0,26 (edad).

Aunque protein_content_of_last_meal también parece muy prometedora, la relación parece curva, no lineal. Dejaremos esta característica por ahora y volveremos a ella en el próximo ejercicio.

R-Cuadrado
Hemos mostrado el valor R-Cuadrado de estos modelos y lo hemos utilizado como medida de "corrección" de nuestra regresión, pero ¿qué es?

Intuitivamente, podemos pensar en R-cuadrado como el cociente de cuánto mejor es nuestra línea de regresión que una regresión ingenua que sólo va directamente a través de la media de todos los ejemplos. A grandes rasgos, el R-cuadrado se calcula tomando la pérdida/error del modelo entrenado y dividiéndola por la pérdida/error del modelo ingenuo. Eso da un rango en el que 0 es mejor y 1 es peor, así que se resta todo de 1 para dar la vuelta a esos resultados.

A continuación, mostramos una vez más el gráfico de dispersión con la edad y la temperatura_del_núcleo, pero esta vez, mostramos dos líneas de regresión. La primera es la línea ingenua que pasa directamente por la media. Esto tiene un R-cuadrado de 0 (ya que no es mejor que sí mismo). Un R-Cuadrado de 1 sería una línea que se ajustara perfectamente a cada ejemplo de entrenamiento. El segundo gráfico muestra nuestra línea de regresión entrenada y una vez más vemos su R-Cuadrado.

In [4]:
formula = "core_temperature ~ age"
age_trained_model = smf.ols(formula = formula, data = dataset).fit()
age_naive_model = smf.ols(formula = formula, data = dataset).fit()
age_naive_model.params[0] = dataset['core_temperature'].mean()
age_naive_model.params[1] = 0

print("naive R-squared:", age_naive_model.rsquared)
print("trained R-squared:", age_trained_model.rsquared)

# Show a graph of the result
graphing.scatter_2D(dataset, label_x="age", 
                                label_y="core_temperature",
                                title = "Naive model",
                                trendline=lambda x: dataset['core_temperature'].repeat(len(x)), 
                                show=True)
# Show a graph of the result
graphing.scatter_2D(dataset, label_x="age", 
                                label_y="core_temperature",
                                title = "Trained model",
                                trendline=lambda x: age_trained_model.params[1] * x + age_trained_model.params[0])



naive R-squared: 0.0
trained R-squared: 0.2648116081342463


## Regresión lineal múltiple 
En lugar de modelarlos por separado, intentemos combinarlos en un solo modelo. Después de todo, la grasa corporal no parecía ser útil, así que usemos el sexo masculino y la edad como características.

In [5]:
model = smf.ols(formula = "core_temperature ~ age + male", data = dataset).fit()

print("R-squared:", model.rsquared)

R-squared: 0.31485126997680113


Al utilizar ambas características al mismo tiempo, obtuvimos un resultado mejor que cualquiera de los modelos de una sola característica (univariante).

¿Cómo podemos ver esto? Pues bien, una regresión lineal simple se dibuja en 2d. Si trabajamos con una variable adicional, añadimos una dimensión y trabajamos en 3D.

In [6]:
import numpy as np
# Show a graph of the result
# this needs to be 3D, because we now have three variables in play: two features and one label

def predict(age, male):
    '''
    This converts given age and male values into a prediction from the model
    '''
    # to make a prediction with statsmodels, we need to provide a dataframe
    # so create a dataframe with just the age and male variables
    df = pandas.DataFrame(dict(age=[age], male=[male]))
    return model.predict(df)

# Create the surface graph
fig = graphing.surface(
    x_values=np.array([min(dataset.age), max(dataset.age)]),
    y_values=np.array([0, 1]),
    calc_z=predict,
    axis_title_x="Age",
    axis_title_y="Male",
    axis_title_z="Core temperature"
)

# Add our datapoints to it and display
fig.add_scatter3d(x=dataset.age, y=dataset.male, z=dataset.core_temperature, mode='markers')
fig.show()

El gráfico anterior es interactivo. Intente girarlo para ver cómo el modelo (mostrado como un plano sólido) predeciría la temperatura central a partir de diferentes combinaciones de edad y sexo.

Inspeccionar nuestro modelo
Cuando tenemos más de dos características, resulta muy difícil visualizar estos modelos. Normalmente tenemos que mirar los parámetros directamente. Hagámoslo ahora. Statsmodels, una de las bibliotecas habituales de aprendizaje automático y estadística, ofrece un método summary() que proporciona información sobre nuestro modelo

In [7]:
# Print summary information
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:       core_temperature   R-squared:                       0.315
Model:                            OLS   Adj. R-squared:                  0.300
Method:                 Least Squares   F-statistic:                     21.83
Date:                Sat, 25 Mar 2023   Prob (F-statistic):           1.58e-08
Time:                        14:00:34   Log-Likelihood:                -85.295
No. Observations:                  98   AIC:                             176.6
Df Residuals:                      95   BIC:                             184.3
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     37.9793      0.135    282.094      0.000      37.712      38.247
age            0.1406      0.026      5.459      0.000       0.089       0.192
male           0.3182      0.121      2.634      0.010       0.078       0.558
==============================================================================
Omnibus:                       21.610   Durbin-Watson:                   2.369
Prob(Omnibus):                  0.000   Jarque-Bera (JB):                5.227
Skew:                           0.121   Prob(JB):                       0.0733
Kurtosis:                       1.895   Cond. No.                         12.9
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Si nos fijamos en la esquina superior derecha, podemos ver nuestra estadística R-cuadrado que imprimimos anteriormente.

Un poco más abajo y a la izquierda, también podemos ver información sobre los datos con los que hemos entrenado nuestro modelo. Por ejemplo, podemos ver que lo hemos entrenado con 98 observaciones (Nº Observaciones).

Debajo encontramos información sobre nuestros parámetros, en una columna llamada coef (que significa coeficientes, un sinónimo de parámetros en el aprendizaje automático). Aquí podemos ver que el intercepto era de aproximadamente 38 - lo que significa que el modelo predice una temperatura central de 38 para un perro con edad=0 y macho=0. Debajo de esto vemos que el parámetro para la edad es 0,14, lo que significa que por cada año adicional de edad, la temperatura predicha aumentaría 0,14 grados centígrados. Para el macho, podemos ver un parámetro de 0,32, lo que significa que el modelo estima que todos los perros (es decir, donde macho == 1) tienen temperaturas 0,32 grados celsius más altas que las hembras (es decir, donde macho == 0).

Aunque no tenemos espacio aquí para entrar en detalles, la columna P también es muy útil. Nos indica el grado de confianza del modelo en el valor de este parámetro. Como regla general, si el valor p es inferior a 0,05, hay muchas posibilidades de que esta relación sea fiable. Por ejemplo, aquí tanto la edad como el sexo masculino son inferiores a 0,05, por lo que podemos confiar en el uso de este modelo en el mundo real.

Como ejercicio final, hagamos lo mismo con nuestro anterior modelo de regresión lineal simple, relacionando la edad con la temperatura_del_corazón. Lee la siguiente tabla y comprueba lo que puedes deducir de este modelo.

In [8]:
age_trained_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:       core_temperature   R-squared:                       0.265
Model:                            OLS   Adj. R-squared:                  0.257
Method:                 Least Squares   F-statistic:                     34.58
Date:                Sat, 25 Mar 2023   Prob (F-statistic):           5.94e-08
Time:                        14:01:08   Log-Likelihood:                -88.749
No. Observations:                  98   AIC:                             181.5
Df Residuals:                      96   BIC:                             186.7
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     38.0879      0.132    288.373      0.000      37.826      38.350
age            0.1533      0.026      5.880      0.000       0.102       0.205
==============================================================================
Omnibus:                       43.487   Durbin-Watson:                   2.492
Prob(Omnibus):                  0.000   Jarque-Bera (JB):                6.605
Skew:                           0.087   Prob(JB):                       0.0368
Kurtosis:                       1.740   Cond. No.                         11.3
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Resumen
Hemos cubierto los siguientes conceptos en este ejercicio:

Construir Modelos de Regresión Lineal Simple y Múltiple.
Comparar el rendimiento de ambos modelos observando los valores de R-Cuadrado.
Inspeccionar los modelos para entender cómo funcionan.